# Iterative Prompt Development


Building a perfect prompt for large language model will always require a re-think for the prompt. There isint a perfect prompt but to get one step closer to the perfection on should work out with different prompts to be satisfied. There's always a process, below is the process for reaching to the prompt you are looking for:

1. Idea
2. Implementation (code/data)
3. Experimental result
4. Error Analysis
5. Maybe change the idea (exactly what problem to solve and how to solve it) ? and repeat again

When working with LLM's and prompt engineering to develop an application the process will contain the looping process where one writes a clear and extensive prompt for the LLM, the prompt gives time to LLM to think (as described in Part 1). Analyze the response and check where it can fail or how to make it more robust and why its not giving the desired output. Now change and refine the idea and prompt accordingly and repeat the loop again ! 

__Prompt guildelines__

- Be clear and specific
- Analyze why result does not give desired output
- Refine the idea and the prompt
- Repeat

> NOTE: Be aware of the articles which says __30 perfect prompt for ChatGPT__ as there isint quite any prefect prompt which can be helpful for specific application.

## OpenAI utilites

In [1]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [3]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response =  openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message.content

## Generate a marketing product description from a product fact sheet

In [4]:
fact_sheet_chair = """
OVERVIEW
- Part of a beautiful family of mid-century inspired office furniture, 
including filing cabinets, desks, bookcases, meeting tables, and more.
- Several options of shell color and base finishes.
- Available with plastic back and front upholstery (SWC-100) 
or full upholstery (SWC-110) in 10 fabric and 6 leather options.
- Base finish options are: stainless steel, matte black, 
gloss white, or chrome.
- Chair is available with or without armrests.
- Suitable for home or business settings.
- Qualified for contract use.

CONSTRUCTION
- 5-wheel plastic coated aluminum base.
- Pneumatic chair adjust for easy raise/lower action.

DIMENSIONS
- WIDTH 53 CM | 20.87”
- DEPTH 51 CM | 20.08”
- HEIGHT 80 CM | 31.50”
- SEAT HEIGHT 44 CM | 17.32”
- SEAT DEPTH 41 CM | 16.14”

OPTIONS
- Soft or hard-floor caster options.
- Two choices of seat foam densities: 
 medium (1.8 lb/ft3) or high (2.8 lb/ft3)
- Armless or 8 position PU armrests 

MATERIALS
SHELL BASE GLIDER
- Cast Aluminum with modified nylon PA6/PA66 coating.
- Shell thickness: 10 mm.
SEAT
- HD36 foam

COUNTRY OF ORIGIN
- Italy
"""

### Attempt 1

Initial attempt

In [18]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

Introducing our stunning mid-century inspired office chair, the perfect addition to any home or business setting. This chair is part of a beautiful family of office furniture, including filing cabinets, desks, bookcases, meeting tables, and more, all designed with a timeless mid-century aesthetic.

One of the standout features of this chair is the variety of customization options available. You can choose from several shell colors and base finishes to perfectly match your existing decor. The chair is available with either plastic back and front upholstery or full upholstery in a range of 10 fabric and 6 leather options, allowing you to create a look that is uniquely yours.

The chair is also available with or without armrests, giving you the flexibility to choose the option that best suits your needs. The base finish options include stainless steel, matte black, gloss white, or chrome, ensuring that you can find the perfect match for your space.

In terms of construction, this chair is

__Observation:__

- The description is too long which might not fit for our need.
- The description should be nice and crisp containig all the essential things which one might buyer wants to look at first glance.

### Attempt 2


Need for specific length

In [19]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

Introducing our mid-century inspired office chair, part of a stunning furniture collection. With various color and finish options, choose between plastic or full upholstery in fabric or leather. The chair features a durable aluminum base with 5 wheels and pneumatic height adjustment. Perfect for home or business use. Made in Italy.


In [20]:
len(response.split(" "))

51

### Attempt 2.1


In [21]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

Use at most 3 sentences.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

Introducing our stunning mid-century inspired office chair, part of a beautiful family of furniture designed for both home and business settings. With a range of shell colors and base finishes to choose from, as well as upholstery options in fabric or leather, this chair offers both style and comfort. Constructed with a 5-wheel plastic coated aluminum base and featuring a pneumatic chair adjust for easy raise/lower action, this chair is not only aesthetically pleasing but also functional. Made in Italy, this chair is the perfect addition to any workspace.


In [24]:
len(response.split(".")) -1 # since the last one will carry an blank character after the last dot

4

### Attempt 2.2

In [27]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

Use at most 250 characters.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

Introducing our mid-century inspired office chair, part of a beautiful furniture collection. With various color and finish options, it offers comfort and style. Suitable for home or business use, it features a 5-wheel base and pneumatic chair adjust. Made in Italy.


In [29]:
len(response)

265

__Observation:__

- The response is very close to 50 words.
- The response is not carrying all the technical details which one might need if the description is gonna read by the retailers.

> Note: LLM's are not great at counting their response characters, words or sentences. They provide close to our response as they have looked over thousands of similar data from their training period and know the repsonse to 250 words restriction allows it to utter specific number of token before uttering the stop word. 

### Analysis over range of words

In [5]:
import json
import time
from tqdm import tqdm_notebook as tqdm

words = [50, 100, 444, 777, 1000]
response_analysis = {}
num_iters = 20

for word in words:
    print(f"\n======> {word =}")
    if not response_analysis.get(word):
        response_analysis[word] = []

    for iter in tqdm(range(num_iters)):
        prompt = f"""
        Your task is to help a marketing team create a 
        description for a retail website of a product based 
        on a technical fact sheet.

        Write a product description based on the information 
        provided in the technical specifications delimited by 
        triple backticks.

        Use at most {word} words.

        Technical specifications: ```{fact_sheet_chair}```
        """

        start = time.time()
        response = get_completion(prompt)
        end = time.time()

        response_analysis[word].append(
            dict(iteration=iter, response=response, time_taken=end - start)
        )

# Convert and write JSON object to file
with open("words-analysis.json", "w") as outfile:
    json.dump(response_analysis, outfile)


======> word =50


/tmp/ipykernel_16888/2048634460.py:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for iter in tqdm(range(num_iters)):


  0%|          | 0/20 [00:00<?, ?it/s]


======> word =100


/tmp/ipykernel_16888/2048634460.py:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for iter in tqdm(range(num_iters)):


...

  0%|          | 0/20 [00:00<?, ?it/s]


======> word =777


  0%|          | 0/20 [00:00<?, ?it/s]


======> word =1000


  0%|          | 0/20 [00:00<?, ?it/s]

In [5]:
import json
with open("words-analysis.json", "r") as f:
    words_analysis_data = json.load(f)


words_analysis_data

{'50': [{'iteration': 0,
   'response': 'Introducing our mid-century inspired office chair, part of a stunning furniture collection. With various color and finish options, choose between plastic or full upholstery in fabric or leather. The chair features a durable base, pneumatic chair adjustment, and is suitable for both home and business use. Made in Italy.',
   'time_taken': 2.5220797061920166},
  {'iteration': 1,
   'response': 'Introducing our mid-century inspired office chair, part of a stunning furniture collection. With various color and finish options, choose between plastic or full upholstery in fabric or leather. The chair features a durable base, pneumatic chair adjustment, and is suitable for both home and business settings. Made in Italy.',
   'time_taken': 5.417208909988403},
  {'iteration': 2,
   'response': 'Introducing our mid-century inspired office chair, part of a stunning furniture collection. With various color and finish options, choose between plastic or full u

In [6]:
words_analysis_data.keys()

dict_keys(['50', '100', '444', '777', '1000'])

In [22]:
words = [50, 100, 444, 777, 1000]
import statistics as st

for word in words:
    word_data = words_analysis_data[str(word)]
    mean = st.mean([ len(wi['response'].split(" ")) for wi in word_data])
    print(f"Question given word limit= {word} ")
    print(f"response mean word length= {mean}")
    print("\n")

Question given word limit= 50 
response mean word length= 49.55


Question given word limit= 100 
response mean word length= 97.85


Question given word limit= 444 
response mean word length= 426.3


Question given word limit= 777 
response mean word length= 507.1


Question given word limit= 1000 
response mean word length= 538.1




__Observation:__

1. As the word limit is increasing the response form the llm is obeying to return the required response lenght.

### Attempt 3

Need of technical flavour

In [32]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

Introducing our mid-century inspired office chair, part of a beautiful furniture collection. With various shell colors and base finishes, it offers versatility for any setting. Choose between plastic or full upholstery in a range of fabric and leather options. The chair features a durable aluminum base with 5-wheel design and pneumatic chair adjustment. Made in Italy.


__Observation:__

- The repsonse is now carrying around 50 words.
- It is containing technical terms like aluminum and pneumatic.
- It also talks about it can be available in plastic, leather or full upholstery.

### Attempt 4

Include specific extraction from input

In [33]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

At the end of the description, include every 7-character 
Product ID in the technical specification.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

Introducing our mid-century inspired office chair, perfect for both home and business settings. This chair is part of a beautiful family of furniture, offering a range of options including filing cabinets, desks, bookcases, and meeting tables. The chair is available with plastic or full upholstery in a variety of fabric and leather options. Choose from stainless steel, matte black, gloss white, or chrome base finishes. With its 5-wheel plastic coated aluminum base and pneumatic chair adjust, this chair offers easy raise/lower action. It also comes with the option of soft or hard-floor casters and two choices of seat foam densities. The chair is constructed with a cast aluminum shell with a modified nylon coating and a 10mm shell thickness. The seat is made with HD36 foam for maximum comfort. Made in Italy, this chair is qualified for contract use. Product ID: SWC-100, SWC-110.


__Observation:__

- Including required things to add in the response will lead to force LLM to include it in its response.
- This can be helpful if the LLM is not consistent producing response and despite prduct id being an important technical detail can miss it to include in its response.

### Attempt 4.1

Format to HTML (or json)

In [37]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

At the end of the description, include every 7-character 
Product ID in the technical specification.

After the description, include a table that gives the 
product's dimensions. The table should have two columns.
In the first column include the name of the dimension. 
In the second column include the measurements in inches only.

Give the table the title 'Product Dimensions'.

Format everything as HTML that can be used in a website. 
Place the description in a <div> element.

Technical specifications: ```{fact_sheet_chair}```
"""

response = get_completion(prompt)
print(response)

<div>
  <h2>Product Description</h2>
  <p>
    Introducing our latest addition to our mid-century inspired office furniture collection - the SWC Chair. This chair is part of a beautiful family of furniture that includes filing cabinets, desks, bookcases, meeting tables, and more. With its sleek design and customizable options, the SWC Chair is the perfect choice for any home or business setting.
  </p>
  <p>
    The SWC Chair is available in several options of shell color and base finishes, allowing you to create a look that matches your style. You can choose between plastic back and front upholstery or full upholstery in a variety of fabric and leather options. The base finish options include stainless steel, matte black, gloss white, or chrome. Additionally, you have the choice of having the chair with or without armrests.
  </p>
  <p>
    Constructed with durability in mind, the SWC Chair features a 5-wheel plastic coated aluminum base, ensuring stability and easy mobility. The chai

In [38]:
from IPython.display import display, HTML
display(HTML(response))

Dimension,Measurement (inches)
Width,"20.87"""
Depth,"20.08"""
Height,"31.50"""
Seat Height,"17.32"""
Seat Depth,"16.14"""


## Observation For ALL

- There isint any good prompt one needs to be refined at each step of the loop.
- This specific use case explains a thin grain from the heap and there are many possible ways to do the same.
- This use case here is for short description sometimes the need is for longer description or with different tone (profession, casual or something custom to ones need).
- Try to play with the ordering of the instructions that can also change the response drastically.
- Most of the time where the application is in its premature stage people tend to try with a single example and later on at mature stage try to evaluate it againts larger set of examples where its capabilites can be assessed by many examples and not just 1.
